<a href="https://colab.research.google.com/github/andrewli4938/TrafficLightDetection/blob/main/Group1_Traffic_Light_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install ultralytics

In [2]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import glob
import shutil
from PIL import Image
from google.colab import drive

from ultralytics import YOLO

In [ ]:
# import our dataset
import kagglehub
source_path = kagglehub.dataset_download("mbornoe/lisa-traffic-light-dataset")

In [ ]:
%pwd
%rm -r data
%mkdir data  # root folder for data

%cd data
%mkdir images
%mkdir labels

%cd images
%mkdir train
%mkdir val
%mkdir test

%cd ../labels
%mkdir train
%mkdir val
%mkdir test

%cd /content

In [ ]:
# these two only have total 1000 images, and have duplicate filenames so we can deal with them later
%mv {source_path}/sample-dayClip6 {source_path}/..
%mv {source_path}/sample-nightClip1 {source_path}/..

In [ ]:
# move all of the filenames into list
image_paths = glob.glob(f"{source_path}/**/*.jpg", recursive=True)

# process the annotations
target = "frameAnnotationsBOX.csv"
annotations = pd.DataFrame()
annotation_paths = glob.glob(f"{source_path}/**/{target}", recursive=True)
for p in annotation_paths:
  # combine the csv files into one
  new_frame = pd.read_csv(p, sep=";")
  new_frame = new_frame.drop(['Origin file', 'Origin frame number', 'Origin track', 'Origin track frame number'], axis=1)
  annotations = pd.concat([annotations, new_frame])
filenames = annotations[annotations.columns[0]].str.split("/").str[-1]
annotations["Filename"] = filenames
annotations.set_index("Filename", inplace=True)
annotations

In [ ]:
def move_files(paths_list, destination_dir):
  for file in paths_list:
    shutil.copy(file, destination_dir)

In [ ]:
# TODO: split the data into train, val, test
image_train, image_val_test = train_test_split(image_paths, train_size=0.7, test_size=0.3, random_state=42, shuffle=True)
image_val, image_test = train_test_split(image_val_test, train_size=0.5, random_state=42, shuffle=True)

print(len(image_train))
print(len(image_val))
print(len(image_test))

image_train_dir = "/content/data/images/train"
image_val_dir = "/content/data/images/val"
image_test_dir = "/content/data/images/test"
move_files(image_train, image_train_dir)
move_files(image_val, image_val_dir)
move_files(image_test, image_test_dir)

print(len(os.listdir(image_train_dir)))
print(len(os.listdir(image_val_dir)))
print(len(os.listdir(image_test_dir)))

In [ ]:
# import yaml file: defines image locations and encodes classes into numbers for the model
!wget https://raw.githubusercontent.com/andrewli4938/TrafficLightDetection/refs/heads/main/data.yaml

In [ ]:
def convert_to_xywh(box_coordinates, image_dimensions):
  image_width, image_height = image_dimensions
  x1, y1, x2, y2 = box_coordinates
  width, height = x2-x1, y2-y1
  x_center = (x1+x2)/2
  y_center = (y1+y2)/2

  x_center = x_center/image_width
  y_center = y_center/image_height
  width = width/image_width
  height = height/image_height

  return (x_center, y_center, width, height)

In [ ]:
"""
The *.txt file should be formatted with one row per object in
class x_center y_center width height format. Box coordinates
must be in normalized xywh format (from 0 to 1). If your boxes
are in pixels, you should divide x_center and width by image width,
and y_center and height by image height. Class numbers should be
zero-indexed (start with 0).
"""
image_dims = (1280, 960)

def create_labels(source_dir, destination_dir, annotations_df, encoding):
  bounding_box_df = annotations_df.iloc[:, 1:]
  for file in os.listdir(source_dir):
    write_path = os.path.join(destination_dir, file.replace(".jpg", ".txt"))
    if file in annotations_df.index:
      rows = annotations_df.loc[file]
      if isinstance(rows, pd.Series):
        rows = pd.DataFrame([rows])
    else:
      continue
    with open(write_path, "w") as image_label:
      for i in range(len(rows)):  # iterate through each label for current image
        class_number = encoding[rows.iloc[i, 0]]
        bounding_box = bounding_box_df.iloc[i]
        # print(bounding_box)
        # print(f"image is: {file} at index {i}")
        # print(type(rows))
        xywh = convert_to_xywh(bounding_box, image_dims)  # calculate bounding box coordinates here
        image_label.write(f"{class_number} {xywh[0]} {xywh[1]} {xywh[2]} {xywh[3]}\n")


In [ ]:
class_encoding = {"stop": 0, "stopLeft": 1, "warning": 2, "warningLeft": 3,
                  "warningLeft": 4, "go": 5, "goForward": 6, "goLeft": 6}

label_train_dir = "/content/data/labels/train"
label_val_dir = "/content/data/labels/val"
label_test_dir = "/content/data/labels/test"

create_labels(image_train_dir, label_train_dir, annotations, class_encoding)
create_labels(image_val_dir, label_val_dir, annotations, class_encoding)
create_labels(image_test_dir, label_test_dir, annotations, class_encoding)

print(len(os.listdir(label_train_dir)))
print(len(os.listdir(label_val_dir)))
print(len(os.listdir(label_test_dir)))

In [3]:
torch.cuda.is_available()

True

In [ ]:
torch.cuda.empty_cache()

In [4]:
!nvidia-smi

Tue Nov  5 03:36:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   50C    P8              13W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!rm -r /content/runs

In [ ]:
# model training
model = YOLO("yolo11n.pt")
results = model.train(data="/content/data.yaml", epochs=40, imgsz=896, batch=64)

100%|██████████| 5.35M/5.35M [00:00<00:00, 64.7MB/s]


Ultralytics 8.3.27 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/data.yaml, epochs=40, time=None, patience=100, batch=64, imgsz=896, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, 

train: Scanning /content/data/labels/train.cache... 25473 images, 4638 backgrounds, 0 corrupt: 100%|██████████| 30111/30111 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/data/labels/val.cache... 5412 images, 1040 backgrounds, 0 corrupt: 100%|██████████| 6452/6452 [00:00<?, ?it/s]


...

                   all       6452      16295       0.82      0.725      0.821      0.674

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      40/40      18.2G     0.7676     0.6597     0.8002         57        896: 100%|██████████| 471/471 [08:28<00:00,  1.08s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 51/51 [00:50<00:00,  1.01it/s]

                   all       6452      16295      0.824      0.723      0.824      0.678



40 epochs completed in 6.423 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.27 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLO11n summary (fused): 238 layers, 2,583,517 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   2%|▏         | 1/51 [00:01<00:50,  1.00s/it]

In [9]:
val_model = YOLO('/content/runs/detect/train/weights/last.pt')
result = val_model.val(data='/content/data.yaml')

Ultralytics 8.3.27 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLO11n summary (fused): 238 layers, 2,583,517 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning /content/data/labels/val.cache... 5412 images, 1040 backgrounds, 0 corrupt: 100%|██████████| 6452/6452 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 404/404 [00:56<00:00,  7.14it/s]


                   all       6452      16295      0.824      0.723      0.824      0.728
                  stop       2792       6704      0.899       0.94      0.974      0.891
              stopLeft       1562       1918      0.873      0.828      0.922      0.834
               warning        150        344      0.817      0.776      0.856      0.765
                    go         39         57      0.696      0.298       0.51      0.409
             goForward       2780       6903       0.89      0.949      0.975      0.889
                goLeft        276        369      0.769      0.549      0.705      0.577
Speed: 0.4ms preprocess, 2.9ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/val2


In [10]:
result.results_dict

{'metrics/precision(B)': 0.8239254840899588,
 'metrics/recall(B)': 0.7234864557570194,
 'metrics/mAP50(B)': 0.8237260585905727,
 'metrics/mAP50-95(B)': 0.7276078642319429,
 'fitness': 0.7372196836678059}

In [ ]:
#TODO: the label files are producing duplicate files, fix the create_labels() function

In [ ]:
"""
OVERVIEW
Basically our LISA dataset contains around 44k images and a bunch of labels

Labels are:
  - light status: {go, slow, stop}
  - bounding box of where the stoplight is xyxy coordinates (top left, bottom right)
  - each image may have many labels since there are many stoplights in a single image

DaySequence
"""

In [ ]:
"""
END HERE
"""

In [ ]:
# # centralize both annotations and images (not scattered in different folders)
# annotations = pd.DataFrame()

# target = "frameAnnotationsBOX.csv"
# annotation_paths = glob.glob(f"{path}/**/{target}", recursive=True)

# # I am going to omit the approximately 1000 pictures and labels in sample-dayClip6
# # and sample-nightClip1 because it is really annoying to work with right now and and
# # we have 43k other values we can work with first
# for p in annotation_paths:
#   tokens = p.split("/")
#   if tokens[-2] == "sample-nightClip1" or tokens[-2] == "sample-dayClip6":  # omit here
#     continue

#   # combine the csv files into one
#   new_frame = pd.read_csv(p, sep=";")
#   new_frame = new_frame.drop(['Origin file', 'Origin frame number', 'Origin track', 'Origin track frame number'], axis=1)
#   annotations = pd.concat([annotations, new_frame])

#   # move all of the nested image files into /content/images_dir
#   if len(tokens)==14:
#     subpath = path+"/"+tokens[-3]+"/"+tokens[-3]+"/"+tokens[-2]+"/frames"
#   else:
#     subpath = path+"/"+tokens[-2]+"/"+tokens[-2]+"/frames"
#   !cp -r {subpath}/* /content/images_dir

# filenames = annotations[annotations.columns[0]].str.split("/").str[-1]
# annotations["Filename"] = filenames
# annotations  # NOTE: our labels are in this df here

In [ ]:
# #TODO: define custom dataset class
# class TrafficLightDataset(Dataset):
#   def __init__(self, annotations_df, img_dir, transform=None, target_transform=None):
#     self.img_labels = annotations_df
#     self.img_dir = img_dir
#     self.transform = transform
#     self.target_transform = target_transform

#   def __len__(self):
#     return len(self.img_labels)

#   def __getitem__(self, idx):
#     img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
#     image = read_image(img_path)
#     label = tuple(self.img_labels.iloc[idx, 1:])
#     if self.transform:
#       image = self.transform(image)
#     if self.target_transform:
#       pass  # not sure how this will work on a tuple (worry about it later)
#     return image, label


In [ ]:
# #TODO: split our data into train and test dataloaders

# img_dir = "content/images_dir"
# dataset = TrafficLightDataset(annotations, img_dir, transform=None, target_transform=None)

# # shuffle our indices before splitting (need day and night in both sets)
# indices = torch.randperm(len(dataset))
# train_size = int(0.8*len(dataset))
# train_indices = indices[:train_size]
# test_indices = indices[train_size:]

# # split into train and test
# train_dataset = torch.utils.data.Subset(dataset, train_indices)
# test_dataset = torch.utils.data.Subset(dataset, test_indices)

# # we have our dataloaders here
# train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=1000, shuffle=True)